# Visualize Network

## Data

In [1]:
import json
import numpy as np
np.set_printoptions( precision=2, edgeitems=6, linewidth=240 )

In [2]:
data_paths = [ "../data/shanghan_formulas.json",  "../data/shanghan_herbs.json" ]
s_fmls_, s_herbs_ = [ json.loads( open(data_path, 'r', encoding='utf-8').read() ) for data_path in  data_paths ]
s_fmls = s_fmls_.get('formulas').items()
fml_list = [ list( fml[1].get('ingOrg').keys() ) for fml in s_fmls ]
fml_corpus = [ " ".join(fml) for fml in fml_list ]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

min_df = 5
vectorizer = CountVectorizer( min_df=min_df )
X = vectorizer.fit_transform( fml_corpus)
herb_names =  vectorizer.get_feature_names()
herb_freq = np.sum( X, 0 ).tolist()[0]
print( X.shape )
print( herb_names )
print( herb_freq )

(219, 34)
['감초', '갱미', '건강', '계지', '과루근', '귤피', '대조', '대황', '도인', '마황', '망초', '모려', '반하', '복령', '부자', '생강', '석고', '세신', '시호', '아교', '오미자', '용골', '인삼', '작약', '정력', '지실', '치자', '택사', '행인', '향시', '황금', '황기', '황련', '후박']
[106, 7, 31, 61, 5, 5, 59, 29, 7, 23, 8, 7, 36, 33, 27, 61, 12, 9, 7, 6, 6, 6, 31, 41, 5, 16, 10, 8, 16, 6, 20, 7, 13, 10]


In [4]:
herb_co = X.T * X

print( "# HERB NETWORK MATRIX according to co-occurrence", herb_co.shape )
print( herb_co.toarray() )

# HERB NETWORK MATRIX according to co-occurrence (34, 34)
[[106   6  19  46   3   1 ...  12   1  12   5   6   3]
 [  6   7   1   1   0   0 ...   0   0   0   0   0   0]
 [ 19   1  31   5   1   0 ...   2   0   6   0   5   0]
 [ 46   1   5  61   2   0 ...   6   0   4   5   1   3]
 [  3   0   1   2   5   0 ...   0   0   2   0   0   0]
 [  1   0   0   0   0   5 ...   0   0   0   0   0   0]
 ...
 [ 12   0   2   6   0   0 ...  16   0   0   0   0   2]
 [  1   0   0   0   0   0 ...   0   6   0   0   0   0]
 [ 12   0   6   4   2   0 ...   0   0  20   0   8   0]
 [  5   0   0   5   0   0 ...   0   0   0   7   0   0]
 [  6   0   5   1   0   0 ...   0   0   8   0  13   0]
 [  3   0   0   3   0   0 ...   2   0   0   0   0  10]]


## Libs

In [5]:
from sklearn.preprocessing import scale

def re_scale( arr, zoom=1, bottom=2):
    arr_scaled = scale( herb_freq, axis=0, with_mean=True, with_std=True, copy=True )
    arr_zoomed = arr_scaled * zoom
    arr_min = np.min( arr_zoomed )
    add_mount = bottom - arr_min
    return arr_zoomed + add_mount

In [6]:
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource

def scatter_plot( embedded, title="", size=4):
    lst = embedded.tolist()
    x, y = zip( *lst )
    
    p = figure( plot_width=600, plot_height=600, title=title )
    p.circle(x, y, size=size, color="navy", alpha=0.5 )

    return p

def scatter_plot_tooltip( embedded, size, labels="", title=""  ):
    lst = embedded.tolist()
    x, y = zip( *lst )
    
    source = ColumnDataSource(data=dict( x=x, y=y, size=size, label=labels ))

    TOOLTIPS = [
        ("label", "@label"),
        ("index", "$index"),
        ("(x,y,size)", "($x, $y,@size)"),
    ]
    
    p = figure( plot_width=600, plot_height=600, title=title, tooltips=TOOLTIPS )
    p.circle('x', 'y', size='size', color="navy", alpha=0.5, source=source)

    return p


## MDS 

In [7]:
C = herb_co.toarray()

n, _ = C.shape

dissimilarities = np.max( C ) - C

dg = np.zeros( n )
np.fill_diagonal( dissimilarities, dg )
print( dissimilarities )

[[  0 100  87  60 103 105 ...  94 105  94 101 100 103]
 [100   0 105 105 106 106 ... 106 106 106 106 106 106]
 [ 87 105   0 101 105 106 ... 104 106 100 106 101 106]
 [ 60 105 101   0 104 106 ... 100 106 102 101 105 103]
 [103 106 105 104   0 106 ... 106 106 104 106 106 106]
 [105 106 106 106 106   0 ... 106 106 106 106 106 106]
 ...
 [ 94 106 104 100 106 106 ...   0 106 106 106 106 104]
 [105 106 106 106 106 106 ... 106   0 106 106 106 106]
 [ 94 106 100 102 104 106 ... 106 106   0 106  98 106]
 [101 106 106 101 106 106 ... 106 106 106   0 106 106]
 [100 106 101 105 106 106 ... 106 106  98 106   0 106]
 [103 106 106 103 106 106 ... 104 106 106 106 106   0]]


In [8]:
from sklearn.manifold import smacof

# morden MDS via smacof
embedded_mds, stress = smacof( dissimilarities )


In [9]:
output_notebook() 
show( scatter_plot_tooltip( embedded_mds, labels=herb_names, size=re_scale( herb_freq, zoom=10, bottom=6 ), title="MDS" ) )

Loading BokehJS ...

## VOSviewer Mapping

In [10]:
import tensorflow as tf
# for reproducibility
tf.set_random_seed( 99 )
    
# Pairwise Euclidean distance ( Squared )
# https://stackoverflow.com/questions/37009647/compute-pairwise-distance-in-a-batch-without-replicating-tensor-in-tensorflow
def pairwise_distance( X ):
    # Pairwise Euclidean distance ( Squared )
    r_ = tf.reduce_sum( X * X, 1 )
    r = tf.reshape( r_, [-1, 1] )
    d_square = r - 2 * tf.matmul( X, tf.transpose( X ) ) + tf.transpose( r )
    return d_square

# Convert upper triangular part of a matrix
# https://stackoverflow.com/questions/41514722/convert-the-strictly-upper-triangular-part-of-a-matrix-into-an-array-in-tensorfl
def upper_triangular_part( X ):
    n, _ = X.shape
    npmask = np.triu( np.ones((n, n), dtype=np.bool_), 1)
    return tf.boolean_mask(X, npmask)

In [11]:
def vos_mapping( arr_sim, learning_rate=1e-4, n_iter=1000, n_dim=2, random_state=99, verbose=True ):
    
    tf.set_random_seed( random_state )
    n, _ = arr_sim.shape

    S = tf.placeholder( tf.float32, [n, n] )
    X = tf.Variable( tf.random_normal( [n, n_dim], seed=random_state ), name='embedded')

    # pairwise distance
    d_square = pairwise_distance( X )

    S_upper_tri = upper_triangular_part( S )
    d_square_tri = upper_triangular_part( d_square )

    # cost function
    c1 = tf.reduce_sum( tf.multiply( S_upper_tri, d_square_tri ) ) 
    c2 = tf.reduce_sum( tf.sqrt( d_square_tri  ) ) 
    cost =  c1 - c2

    train = tf.train.GradientDescentOptimizer( learning_rate=learning_rate ).minimize( cost )
    
    # Launch the graph in a session.
    sess = tf.Session()
    sess.run( tf.global_variables_initializer() )

    for step in range( n_iter ):
        _, cost_val, c1_v, c2_v = sess.run([ train, cost, c1, c2 ], feed_dict={ S: arr_sim } )
        if verbose:
            if step % (n_iter // 10) == 0: 
                print( "Step {:05d}  ==> Cost: {} .. [ {} {} ]".format( step, cost_val, c1_v, c2_v ) )
                
    embedded = sess.run( X, feed_dict={ S: arr_sim } )
    return embedded


## With Co-occurrence (C)

In [12]:
C = herb_co.toarray()

n, _ = C.shape

dg = np.zeros( n )

Co = C.astype(np.float32)
np.fill_diagonal( Co, dg )
print( "# Co-occurrence Matrix: ")
print( Co )

Co_edge_w = np.sum(Co, 0)
print( "# sum_of_edge: ", Co_edge_w )

# Co-occurrence Matrix: 
[[ 0.  6. 19. 46.  3.  1. ... 12.  1. 12.  5.  6.  3.]
 [ 6.  0.  1.  1.  0.  0. ...  0.  0.  0.  0.  0.  0.]
 [19.  1.  0.  5.  1.  0. ...  2.  0.  6.  0.  5.  0.]
 [46.  1.  5.  0.  2.  0. ...  6.  0.  4.  5.  1.  3.]
 [ 3.  0.  1.  2.  0.  0. ...  0.  0.  2.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0. ...  0.  0.  0.  0.  0.  0.]
 ...
 [12.  0.  2.  6.  0.  0. ...  0.  0.  0.  0.  0.  2.]
 [ 1.  0.  0.  0.  0.  0. ...  0.  0.  0.  0.  0.  0.]
 [12.  0.  6.  4.  2.  0. ...  0.  0.  0.  0.  8.  0.]
 [ 5.  0.  0.  5.  0.  0. ...  0.  0.  0.  0.  0.  0.]
 [ 6.  0.  5.  1.  0.  0. ...  0.  0.  8.  0.  0.  0.]
 [ 3.  0.  0.  3.  0.  0. ...  2.  0.  0.  0.  0.  0.]]
# sum_of_edge:  [386.  21.  99. 243.  22.  12. 274.  79.  12.  88.  25.  35. 147.  99.  69. 266.  47.  41.  50.  13.  32.  25. 131. 166.   8.  43.  16.  18.  62.  10.  98.  25.  42.  34.]


In [13]:
embedded = vos_mapping( Co, n_iter = 2001, learning_rate=1e-3 )

output_notebook() 
show( scatter_plot_tooltip( embedded, labels=herb_names, size=re_scale( Co_edge_w, zoom=10, bottom=6 ), title="VOSviewer" ) )

Step 00000  ==> Cost: 4699.09326171875 .. [ 5759.21484375 1060.12158203125 ]
Step 00200  ==> Cost: -273.7603454589844 .. [ 259.1097717285156 532.8701171875 ]
Step 00400  ==> Cost: -277.94775390625 .. [ 276.2105712890625 554.1583251953125 ]
Step 00600  ==> Cost: -278.81097412109375 .. [ 278.36981201171875 557.1807861328125 ]
Step 00800  ==> Cost: -279.6596984863281 .. [ 279.4372253417969 559.096923828125 ]
Step 01000  ==> Cost: -280.078369140625 .. [ 279.88385009765625 559.9622192382812 ]
Step 01200  ==> Cost: -280.5416564941406 .. [ 280.3057556152344 560.847412109375 ]
Step 01400  ==> Cost: -281.34588623046875 .. [ 281.1712646484375 562.5171508789062 ]
Step 01600  ==> Cost: -281.77447509765625 .. [ 281.59149169921875 563.365966796875 ]
Step 01800  ==> Cost: -282.1150207519531 .. [ 281.9698791503906 564.0848999023438 ]
Step 02000  ==> Cost: -282.374267578125 .. [ 282.27264404296875 564.6469116210938 ]


Loading BokehJS ...

## With Association Strength (S)

In [14]:
margin_x = np.sum( C, 1 )
margin_y = np.sum( C, 0 )
sum_all = np.sum( margin_x )

margin_x_mx = np.repeat( margin_x, n ).reshape( -1, n )
margin_y_mx = np.repeat( margin_y, n ).reshape( n, -1 ).T

S_ = sum_all * C 
base= np.multiply( margin_x_mx,  margin_y_mx )

S = np.divide( S_, base )
print( "# Association Strength Matrix: ")
print( S )

S_edge_w = np.sum(S, 0)
print( "# sum_of_edge: ", S_edge_w )

# Association Strength Matrix: 
[[ 1.52  1.51  1.03  1.07  0.78  0.42 ...  1.09  0.44  0.72  1.1   0.77  0.48]
 [ 1.51 31.    0.95  0.41  0.    0.   ...  0.    0.    0.    0.    0.    0.  ]
 [ 1.03  0.95  6.37  0.44  0.99  0.   ...  0.68  0.    1.36  0.    2.43  0.  ]
 [ 1.07  0.41  0.44  2.29  0.85  0.   ...  0.88  0.    0.39  1.78  0.21  0.78]
 [ 0.78  0.    0.99  0.85 23.81  0.   ...  0.    0.    2.18  0.    0.    0.  ]
 [ 0.42  0.    0.    0.    0.   60.07 ...  0.    0.    0.    0.    0.    0.  ]
 ...
 [ 1.09  0.    0.68  0.88  0.    0.   ...  9.13  0.    0.    0.    0.    2.02]
 [ 0.44  0.    0.    0.    0.    0.   ...  0.   81.38  0.    0.    0.    0.  ]
 [ 0.72  0.    1.36  0.39  2.18  0.   ...  0.    0.    4.99  0.    4.28  0.  ]
 [ 1.1   0.    0.    1.78  0.    0.   ...  0.    0.    0.   23.73  0.    0.  ]
 [ 0.77  0.    2.43  0.21  0.    0.   ...  0.    0.    4.28  0.   14.92  0.  ]
 [ 0.48  0.    0.    0.78  0.    0.   ...  2.02  0.    0.    0.    0.   17.93]]
# sum_of_edge:

In [15]:
embedded = vos_mapping( S, n_iter = 2001, learning_rate=1e-3 )

output_notebook() 
show( scatter_plot_tooltip( embedded, labels=herb_names, size=re_scale( S_edge_w, zoom=10, bottom=6 ), title="VOSviewer" ) )

Step 00000  ==> Cost: 1405.136474609375 .. [ 2465.258056640625 1060.12158203125 ]
Step 00200  ==> Cost: -421.73590087890625 .. [ 375.06903076171875 796.804931640625 ]
Step 00400  ==> Cost: -455.03948974609375 .. [ 441.9765625 897.0160522460938 ]
Step 00600  ==> Cost: -458.3951416015625 .. [ 456.380859375 914.7760009765625 ]
Step 00800  ==> Cost: -458.8274230957031 .. [ 458.2619934082031 917.0894165039062 ]
Step 01000  ==> Cost: -458.8846435546875 .. [ 458.67919921875 917.5638427734375 ]
Step 01200  ==> Cost: -458.89117431640625 .. [ 458.8070068359375 917.6981811523438 ]
Step 01400  ==> Cost: -458.89202880859375 .. [ 458.85650634765625 917.74853515625 ]
Step 01600  ==> Cost: -458.8922119140625 .. [ 458.876708984375 917.7689208984375 ]
Step 01800  ==> Cost: -458.89227294921875 .. [ 458.88543701171875 917.7777099609375 ]
Step 02000  ==> Cost: -458.8924255371094 .. [ 458.8893127441406 917.78173828125 ]


Loading BokehJS ...